In [ ]:
import sympy
from gekko import GEKKO

input_strings = ["(x - y)**0.5 + x - 1", "sin(x - y) + x*y - 1*exp(2)"]

m = GEKKO()

# Define variables
x = m.Var()
y = m.Var()

equations = []
for input_string in input_strings:
    equation_expression = sympy.parse_expr(input_string)
    equation_lambda = sympy.lambdify((x, y), equation_expression)
    equation = m.Equation(equation_lambda(x.value, y.value))
    equations.append(equation)

m.Equations(equations)

In [ ]:
from gekko import GEKKO
from sympy import *
m = GEKKO()
x = m.Var()
y = m.Var()

m.Equations([sin(x*y) - x+y**2 == 1,
             x + y == 0.5
             ])

m.solve(disp=False)
print(x.value, y.value)

In [2]:
import sympy
from gekko import GEKKO

def sympy_to_gekko(expr):
    """Convert SymPy expression to Gekko expression"""
    gekko_functions = {
        sympy.sin: m.sin,
        sympy.cos: m.cos,
        sympy.tan: m.tan,
        sympy.exp: m.exp,
        sympy.log: m.log,
        sympy.sqrt: m.sqrt,
    }

    if expr.func in gekko_functions:
        # Apply the corresponding Gekko function to the arguments
        return gekko_functions[expr.func](*[sympy_to_gekko(arg) for arg in expr.args])
    elif isinstance(expr, sympy.Number):
        # Convert SymPy number to float
        return float(expr)
    elif isinstance(expr, sympy.Symbol):
        # Convert SymPy symbol to Gekko variable
        return gekko_vars[expr.name]
    else:
        raise ValueError(f"Cannot convert expression: {expr}")

# Define the variables in the Gekko model
variables = [sympy.Symbol("x"), sympy.Symbol("y")]
gekko_vars = {var_name: m.Var(name=var_name.name) for var_name in variables}

# Define the function to minimize
expr = (1 - variables[0])**2 + 100*(variables[1] - variables[0]**2)**2

# Convert sympy expression to Gekko expression
gekko_expr = sympy_to_gekko(expr)

# Create a Gekko model
m = GEKKO()

# Define the objective function
m.Obj(gekko_expr)

# Set the solver options
m.options.IMODE = 3

# Solve the optimization problem
m.solve(disp=True)

# Print the results
print(f"x = {gekko_vars['x'].value}")
print(f"y = {gekko_vars['y'].value}")


ValueError: Cannot convert expression: (1 - x)**2 + 100*(-x**2 + y)**2

# Don't edit

In [10]:
import sympy
from gekko import GEKKO

# Input string expressions
input_strings = ["(x - y)**(1/2) + x = 1", "sin(x - y) + x*y - exp(2) = 0"]

# Create a Gekko model
m = GEKKO()

# Define the variables (assuming x and y are the only variables)
x, y = [m.Var() for _ in range(2)]

# Convert input strings to sympy expressions
sympy_exprs = []
for eq in input_strings:
    try:
        sympy_exprs.append(sympy.sympify(eq.replace('e', 'E').replace('^', '**').replace('=', '-')))
    except sympy.SympifyError as e:
        print(f"Error: {e}")
        print(f"Expression: {eq}")

# Define the Gekko equivalent of the sympy functions
gekko_functions = {
    sympy.sin: m.sin,
    sympy.cos: m.cos,
    sympy.tan: m.tan,
    sympy.exp: m.exp,
    sympy.log: m.log,
    sympy.sqrt: m.sqrt,
}

# Convert sympy expressions to Gekko expressions
def sympy_to_gekko(expr):
    if expr.is_Atom:
        if expr.is_Symbol:
            return x if expr.name == 'x' else y
        else:
            return expr
    else:
        if expr.func in gekko_functions:
            gekko_func = gekko_functions[expr.func]
            return gekko_func(*[sympy_to_gekko(arg) for arg in expr.args])
        elif expr.func == sympy.Pow:
            return m.exp(sympy_to_gekko(expr.args[1]) * m.log(sympy_to_gekko(expr.args[0])))
        elif expr.func == sympy.Add:
            return sum([sympy_to_gekko(arg) for arg in expr.args])
        elif expr.func == sympy.Mul:
            result = sympy_to_gekko(expr.args[0])
            for arg in expr.args[1:]:
                result *= sympy_to_gekko(arg)
            return result
        else:
            try:
                return expr.func(*[sympy_to_gekko(arg) for arg in expr.args])
            except Exception as e:
                print(f"Error: {e}")
                print(f"Unsupported expression: {expr}")
                raise


gekko_exprs = [sympy_to_gekko(expr) for expr in sympy_exprs]

# Add equations to the Gekko model
for expr in gekko_exprs:
    m.Equation(expr == 0)


# Solve the system of equations
m.solve()

# Print the solution
print(f"x: {x.value[0]}, y: {y.value[0]}")


apm 103.27.8.110_gk_model6 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            2
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              2
 Number of total equations: -            2
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [8]:
from gekko import GEKKO
import sympy
import re


class Solution():
    def __init__(self, equations, num_variables):
        self.equations = []
        self.variables = []
        self.num_variables = num_variables
        self.num_equations = 0
        self.get_variables(equations)
        self.solution(equations)
        
        
    def get_variables(self, equations):
        var_pattern = re.compile(r'\b[a-zA-Z]+\b')
        variables = []
        for i in range(len(equations)):
            variables = variables + var_pattern.findall(equations[i])
        self.variables = list(set(variables))
        to_remove = ['sin', 'cos', 'tan', 'sqrt', 'log', 'ln', 'exp', 'pi', 'e']
        for i in range(len(to_remove)):
            if to_remove[i] in self.variables:
                self.variables.remove(to_remove[i])
        
    def solution(self, equations):        
        # Create a Gekko model
        m = GEKKO()

        # Define the variables (assuming x and y are the only variables)
        
        input_strings = equations
        variable_names = []
        for i in range(len(self.variables)):
            variable_names.append(str(self.variables[i]))
            
        variables = [m.Var() for _ in range(len(variable_names))]
        variable_dict = dict(zip(variable_names, variables))
        
        # Convert input strings to sympy expressions
        sympy_exprs = []
        for eq in input_strings:
            try:
                sympy_exprs.append(sympy.sympify(eq.replace('=', '-')))
            except sympy.SympifyError as e:
                print(f"Error: {e}")
                print(f"Expression: {eq}")

        # Define the Gekko equivalent of the sympy functions
        gekko_functions = {
            sympy.sin: m.sin,
            sympy.cos: m.cos,
            sympy.tan: m.tan,
            sympy.exp: m.exp,
            sympy.log: m.log,
            sympy.sqrt: m.sqrt,
        }

        # Convert sympy expressions to Gekko expressions
        def sympy_to_gekko(expr, variables):
            if expr.is_Atom:
                if expr.is_Symbol:
                    return variable_dict[expr.name]
                else:
                    return expr
            else:
                if expr.func in gekko_functions:
                    gekko_func = gekko_functions[expr.func]
                    return gekko_func(*[sympy_to_gekko(arg, variables) for arg in expr.args])
                elif expr.func == sympy.Pow:
                    return m.exp(sympy_to_gekko(expr.args[1], variables) * m.log(sympy_to_gekko(expr.args[0], variables)))
                elif expr.func == sympy.Add:
                    return sum([sympy_to_gekko(arg, variables) for arg in expr.args])
                elif expr.func == sympy.Mul:
                    result = sympy_to_gekko(expr.args[0], variables)
                    for arg in expr.args[1:]:
                        result *= sympy_to_gekko(arg, variables)
                    return result
                else:
                    try:
                        return expr.func(*[sympy_to_gekko(arg, variables) for arg in expr.args])
                    except Exception as e:
                        print(f"Error: {e}")
                        print(f"Unsupported expression: {expr}")
                        raise


        gekko_exprs = [sympy_to_gekko(expr, variables) for expr in sympy_exprs]

        # Add equations to the Gekko model
        for expr in gekko_exprs:
            m.Equation(expr == 0)

        # Solve the system of equations
        m.solve()

        # Print the solution
        for var in variables:
            print(f"{var.name}: {var.value[0]}")

In [9]:
input_strings = ["(x-y)**2 + x == 0.5", "sin(x-y) + x*y - exp(2) == 0"]
num_variables = 2
s = Solution(input_strings, num_variables)

apm 103.27.8.110_gk_model5 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            2
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              2
 Number of total equations: -            2
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [6]:
import sympy
from gekko import GEKKO

# Input string expressions
input_strings = ["(x-y)**2 + x == 0.5", "sin(x-y) + x*y - exp(2) == 0"]

# Create a Gekko model
m = GEKKO()

# Define the variables
variable_names = ["x", "y"]
variables = [m.Var() for _ in variable_names]
variable_dict = dict(zip(variable_names, variables))

# Convert input strings to sympy expressions
sympy_exprs = []
for eq in input_strings:
    try:
        sympy_exprs.append(sympy.sympify(eq.replace('=', '-')))
    except sympy.SympifyError as e:
        print(f"Error: {e}")
        print(f"Expression: {eq}")

# Define the Gekko equivalent of the sympy functions
gekko_functions = {
    sympy.sin: m.sin,
    sympy.cos: m.cos,
    sympy.tan: m.tan,
    sympy.exp: m.exp,
    sympy.log: m.log,
    sympy.sqrt: m.sqrt,
}

# Convert sympy expressions to Gekko expressions
def sympy_to_gekko(expr, variables):
    if expr.is_Atom:
        if expr.is_Symbol:
            return variable_dict[expr.name]
        else:
            return expr
    else:
        if expr.func in gekko_functions:
            gekko_func = gekko_functions[expr.func]
            return gekko_func(*[sympy_to_gekko(arg, variables) for arg in expr.args])
        elif expr.func == sympy.Pow:
            return m.exp(sympy_to_gekko(expr.args[1], variables) * m.log(sympy_to_gekko(expr.args[0], variables)))
        elif expr.func == sympy.Add:
            return sum([sympy_to_gekko(arg, variables) for arg in expr.args])
        elif expr.func == sympy.Mul:
            result = sympy_to_gekko(expr.args[0], variables)
            for arg in expr.args[1:]:
                result *= sympy_to_gekko(arg, variables)
            return result
        else:
            try:
                return expr.func(*[sympy_to_gekko(arg, variables) for arg in expr.args])
            except Exception as e:
                print(f"Error: {e}")
                print(f"Unsupported expression: {expr}")
                raise


gekko_exprs = [sympy_to_gekko(expr, variables) for expr in sympy_exprs]

# Add equations to the Gekko model
for expr in gekko_exprs:
    m.Equation(expr == 0)

# Solve the system of equations
m.solve()

# Print the solution
for var in variables:
    print(f"{var.name}: {var.value[0]}")

apm 103.27.8.110_gk_model4 <br><pre> ----------------------------------------------------------------
 APMonitor, Version 1.0.1
 APMonitor Optimization Suite
 ----------------------------------------------------------------
 
 
 
 --------- APM Model Size ------------
 Each time step contains
   Objects      :            0
   Constants    :            0
   Variables    :            2
   Intermediates:            0
   Connections  :            0
   Equations    :            2
   Residuals    :            2
 
 Number of state variables:              2
 Number of total equations: -            2
 Number of slack variables: -            0
 ---------------------------------------
 Degrees of freedom       :              0
 
 **********************************************
 Steady State Optimization with Interior Point Solver
 **********************************************
  
  
 Info: Exact Hessian

******************************************************************************
This program co

In [3]:
from gekko import GEKKO
import sympy
import re

class Solution():
    def __init__(self):
        self.equations = []
        self.variables = []
        self.num_equations = 0

    def get_variables(self, equations):
        var_pattern = re.compile(r'\b[a-zA-Z]+\b')
        variables = []
        for i in range(len(equations)):
            variables = variables + var_pattern.findall(equations[i])
        self.variables = list(set(variables))
        to_remove = ['sin', 'cos', 'tan', 'sqrt',
                     'log', 'ln', 'exp', 'pi', 'e']
        for i in range(len(to_remove)):
            if to_remove[i] in self.variables:
                self.variables.remove(to_remove[i])

    def solution(self, equations, num_variables):
        m = GEKKO(remote=False)
        m.options.SOLVER=1
        var_names = []
        self.get_variables(equations)
        for i in range(len(self.variables)):
            var_names.append(str(self.variables[i]))
        gekko_vars = {name: m.Var(1) for name in var_names}
        sympy_exprs = []
        input_strings = equations
        for eq in input_strings:
            try:
                sympy_exprs.append(sympy.sympify(eq.replace(
                    'e', 'E').replace('^', '**').replace('=', '-')))
            except sympy.SympifyError as e:
                print(eq)
                return None
        gekko_functions = {
            sympy.sin: m.sin,
            sympy.cos: m.cos,
            sympy.tan: m.tan,
            sympy.exp: m.exp,
            sympy.log: m.log,
            sympy.sqrt: m.sqrt,
        }

        def sympy_to_gekko(expr):
            if expr.is_Atom:
                if expr.is_Symbol:
                    return gekko_vars[expr.name]
                else:
                    return expr
            else:
                if expr.func in gekko_functions:
                    gekko_func = gekko_functions[expr.func]
                    return gekko_func(*[sympy_to_gekko(arg) for arg in expr.args])
                elif expr.func == sympy.Pow:
                    return m.exp(sympy_to_gekko(expr.args[1]) * m.log(sympy_to_gekko(expr.args[0])))
                elif expr.func == sympy.Add:
                    return sum([sympy_to_gekko(arg) for arg in expr.args])
                elif expr.func == sympy.Mul:
                    result = sympy_to_gekko(expr.args[0])
                    for arg in expr.args[1:]:
                        result *= sympy_to_gekko(arg)
                    return result
                else:
                    try:
                        return expr.func(*[sympy_to_gekko(arg) for arg in expr.args])
                    except Exception:
                      print(f"Error: {e}")
                      print(f"Unsupported expression: {expr}")
                      raise
        gekko_exprs = [sympy_to_gekko(expr) for expr in sympy_exprs]
        print(gekko_exprs)
        print(gekko_vars)
        for expr in gekko_exprs:
            m.Equation(expr == 0)
        m.solve(disp=True)
        return [gekko_vars[var_name][0] for var_name in var_names]


In [4]:
equations = [
    "5-(100+23*9)*abc+23*9*b-23*20*abc**2+23*20*b**2 = 0",
    "100*abc-(100+23*9)*b+23*9*c-23*20*b**2+23*20*c**2 = 0",
    "100*b-(100+23*9)*c+23*9*d-23*20*c**2+23*20*d**2 = 0",
    "100*c-(100+23*9)*d+23*9*k-23*20*d**2+23*20*k**2 = 0",
    "100*d-(100+23*9)*k+23*9*f-23*20*k**2+23*20*f**2 = 0",
    "100*k-(100+23*9)*f+23*9*g-23*20*f**2+23*20*g**2 = 0",
    "0.039+100*f+(-100+9*23)*g-9*10*h+20*23*g**2-20*10*h**2 = 0",
    "100*g-(100+9*10)*h-9*10*i-20*10*h**2-20*10*i**2 = 0",
    "100*h-(100+9*10)*i-9*10*j-20*10*i**2-20*10*j**2 = 0",
    "100*i-(100+9*10)*j-20*10*j**2 = 0"
]

s = Solution()
print(s.solution(equations, len(equations)))

[((((5+((-460)*(exp(((2)*(log(v6)))))))+((-307)*(v6)))+((207)*(v1)))+((460)*(exp(((2)*(log(v1))))))), (((((0+((-460)*(exp(((2)*(log(v1)))))))+((-307)*(v1)))+((100)*(v6)))+((207)*(v7)))+((460)*(exp(((2)*(log(v7))))))), (((((0+((-460)*(exp(((2)*(log(v7)))))))+((-307)*(v7)))+((100)*(v1)))+((207)*(v9)))+((460)*(exp(((2)*(log(v9))))))), (((((0+((-460)*(exp(((2)*(log(v9)))))))+((-307)*(v9)))+((100)*(v7)))+((207)*(v2)))+((460)*(exp(((2)*(log(v2))))))), (((((0+((-460)*(exp(((2)*(log(v2)))))))+((-307)*(v2)))+((100)*(v9)))+((207)*(v4)))+((460)*(exp(((2)*(log(v4))))))), (((((0+((-460)*(exp(((2)*(log(v4)))))))+((-307)*(v4)))+((100)*(v2)))+((207)*(v8)))+((460)*(exp(((2)*(log(v8))))))), (((((0.0390000000000000+((-200)*(exp(((2)*(log(v10)))))))+((-90)*(v10)))+((100)*(v4)))+((107)*(v8)))+((460)*(exp(((2)*(log(v8))))))), (((((0+((-200)*(exp(((2)*(log(v10)))))))+((-200)*(exp(((2)*(log(v5)))))))+((-190)*(v10)))+((-90)*(v5)))+((100)*(v8))), (((((0+((-200)*(exp(((2)*(log(v5)))))))+((-200)*(exp(((2)*(log(v3

Exception: @error: Solution Not Found
